# Wikishop

**Цель проекта:**
Построить модель, которая будет искать токсичные комментарии и отправлять их на модерацию.

**План проекта:**
1. Загрузка данных;
2. Подготовка данных;
3. Обучение моделей;
4. Выбор лучшей модели;
5. Общие выводы.

## Загрузка данных

In [1]:
# установка бибилеотек

!pip install catboost
!pip install dill

     |████████████████████████████████| 116 kB 2.2 MB/s eta 0:00:01


In [2]:
# импорт бибилеотек

import pandas as pd
import numpy as np
import torch
import transformers
import re
import time
import nltk
import dill as pickle

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk import pos_tag
from nltk.corpus import wordnet

from joblib import Parallel, delayed

from sklearn.feature_extraction.text import TfidfVectorizer

from tqdm import notebook

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, make_scorer

from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')


RANDOM_STATE = 42
TEST_SIZE = 0.25

In [3]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [4]:
# чтение данных
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv', index_col=[0])

data.sort_index(inplace=True)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [6]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

### Выводы:
- Данные не сбалансированы

## Подготовка данных

In [8]:
# Определение функции для преобразования POS-тегов
def get_wordnet_pos(tag):
    tag = tag[0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [9]:
# Функция для лемматизации текста
def lemmatize(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)

    # Получаем POS-теги для всех токенов
    pos_tags = nltk.pos_tag(tokens)

    # Лемматизация с использованием POS-тегов
    lemm_text = [
        lemmatizer.lemmatize(token, get_wordnet_pos(tag))
        for token, tag in pos_tags
    ]

    # Очистка текста от символов
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', ' '.join(lemm_text))
    return " ".join(cleared_text.split())

In [10]:
# Применение параллельной обработки к каждому тексту
def parallel_lemmatization(texts):
    return Parallel(n_jobs=-1, backend="multiprocessing", verbose=5)(delayed(lemmatize)(text) for text in texts)

In [12]:
lemmatizer = nltk.WordNetLemmatizer()

# Применение лемматизации к текстам с использованием параллельной обработки
data['lemm_text'] = parallel_lemmatization(data['text'].tolist())

# Удаление исходного столбца 'text'
data = data.drop(columns=['text'])

# Проверка результата
data.head()

[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.3s finished


,toxic,lemm_text
0,0,explanation why the edits make under my userna...
1,0,d aww he match this background colour i m seem...
2,0,hey man i m really not try to edit war it s ju...
3,0,more i ca n t make any real suggestion on impr...
4,0,you sir be my hero any chance you remember wha...


In [25]:
# Получение стоп-слов и преобразование в список
stopwords = list(nltk_stopwords.words('english'))

# Инициализация TfidfVectorizer с корректным списком стоп-слов
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [26]:
# Разделение данных
X = data.drop(columns=['toxic'])
y = data['toxic']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Применение TF-IDF векторизации к обучающей и тестовой выборке
X_train = count_tf_idf.fit_transform(X_train['lemm_text'])
X_test = count_tf_idf.transform(X_test['lemm_text'])

## Обучение моделей

### LogisticRegression

In [27]:
# Определение модели логистической регрессии с учетом дисбаланса классов
model_lr = LogisticRegression(class_weight='balanced')

# Гиперпараметры для поиска
hyperparams = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'C': [0.1, 1, 10]
}

# GridSearchCV с метрикой F1
clf_lr = GridSearchCV(
    model_lr,
    hyperparams,
    scoring='f1',  # Используем F1 как метрику для оценки модели
    cv=5
)

# Обучение модели
clf_lr.fit(X_train, y_train)

# Вывод лучших гиперпараметров
print("Лучшие гиперпараметры:")
print(clf_lr.best_params_)

# Лучшая модель
model_lr = clf_lr.best_estimator_

# Оценка F1-метрики
f1_lr = clf_lr.best_score_

print('F1 для LogisticRegression =', f1_lr.round(2))

Лучшие гиперпараметры:
{'C': 10, 'solver': 'liblinear'}
F1 для LogisticRegression = 0.76


### DecisionTreeClassifier

In [30]:
# Определение модели дерева решений с учетом дисбаланса классов
model_dtc = DecisionTreeClassifier(
    random_state=RANDOM_STATE,
    class_weight='balanced'
)

# Расширенный набор гиперпараметров для поиска
hyperparams = {
    'max_depth': [5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2'],
    'criterion': ['gini', 'entropy']
}

# RandomizedSearchCV с метрикой F1
clf_dtc = RandomizedSearchCV(
    model_dtc,
    hyperparams,
    scoring='f1',
    cv=5,
    n_iter=10,
    n_jobs=-1,
    random_state=RANDOM_STATE
)

# Обучение модели
clf_dtc.fit(X_train, y_train)

# Вывод лучших гиперпараметров
print("Лучшие гиперпараметры:")
print(clf_dtc.best_params_)

# Лучшая модель
model_dtc = clf_dtc.best_estimator_

# Оценка F1-метрики
f1_dtc = clf_dtc.best_score_

print('F1 для DecisionTreeClassifier =', f1_dtc.round(2))

Лучшие гиперпараметры:
{'min_samples_split': 2, 'min_samples_leaf': 4, 'max_depth': 20}
F1 для DecisionTreeClassifier = 0.6


### CatBoostClassifier

In [36]:
# Инициализация модели CatBoostClassifier
model_cbc = CatBoostClassifier(verbose=False, iterations=100)

# Обучение модели
model_cbc.fit(X_train, y_train)

# Предсказание на тестовых данных
prediction_cbc = model_cbc.predict(X_test)

# Оценка F1-метрики
f1_cbc = f1_score(y_test, prediction_cbc)

# Вывод F1-метрики
print('F1 для CatBoostClassifier =', f1_cbc.round(2))

F1 для CatBoostClassifier = 0.73


## Выбор лучшей модели

In [39]:
# Функция для измерения времени выполнения кода
def measure_time(func):
    start = time.time()
    result = func()
    end = time.time()
    return result, (end - start)

# Оценка моделей
models_results = {}
f1_scorer = make_scorer(f1_score)

In [40]:
# LogisticRegression
train_time_lr = measure_time(lambda: model_lr.fit(X_train, y_train))

# Кросс-валидационная оценка модели
f1_scores = cross_val_score(model_lr, X_train, y_train, cv=5, scoring=f1_scorer)

# Среднее значение F1 с кросс-валидацией
f1_lr = f1_scores.mean()

# Измерение времени предсказания на обучающих данных
pred_time_lr = measure_time(lambda: cross_val_predict(model_lr, X_train, y_train, cv=5))

# Запись результатов модели
models_results['LogisticRegression'] = {
    'model': model_lr,
    'f1': f1_lr,
    'train_time': train_time_lr,
    'pred_time': pred_time_lr
}

# Вывод результатов
print(f'Среднее значение F1 для LogisticRegression по кросс-валидации: {f1_lr:.2f}')

Среднее значение F1 для LogisticRegression по кросс-валидации: 0.76


In [41]:
# DecisionTreeClassifier
train_time_dtc = measure_time(lambda: model_dtc.fit(X_train, y_train))

# Кросс-валидационная оценка модели
f1_scores = cross_val_score(model_dtc, X_train, y_train, cv=5, scoring=f1_scorer)

# Среднее значение F1 с кросс-валидацией
f1_dtc = f1_scores.mean()

# Измерение времени предсказания на обучающих данных
pred_time_dtc = measure_time(lambda: cross_val_predict(model_dtc, X_train, y_train, cv=5))

# Запись результатов модели
models_results['DecisionTreeClassifier'] = {
    'model': model_dtc,
    'f1': f1_dtc,
    'train_time': train_time_dtc,
    'pred_time': pred_time_dtc
}

# Вывод результатов
print(f'Среднее значение F1 для DecisionTreeClassifier по кросс-валидации: {f1_dtc:.2f}')

Среднее значение F1 для DecisionTreeClassifier по кросс-валидации: 0.60


In [42]:
# CatBoostClassifier
train_time_cbc = measure_time(lambda: model_cbc.fit(X_train, y_train))

# Кросс-валидационная оценка модели
f1_scores = cross_val_score(model_cbc, X_train, y_train, cv=5, scoring=f1_scorer)

# Среднее значение F1 с кросс-валидацией
f1_cbc = f1_scores.mean()

# Измерение времени предсказания на обучающих данных
pred_time_cbc = measure_time(lambda: cross_val_predict(model_cbc, X_train, y_train, cv=5))

# Запись результатов модели
models_results['CatBoostClassifier'] = {
    'model': model_cbc,
    'f1': f1_cbc,
    'train_time': train_time_cbc,
    'pred_time': pred_time_cbc
}

# Вывод результатов
print(f'Среднее значение F1 для CatBoostClassifier по кросс-валидации: {f1_cbc:.2f}')

Среднее значение F1 для CatBoostClassifier по кросс-валидации: 0.73


In [46]:
# Сравнение моделей
def score(model):
    return (
        # Нормализация F1: отношение F1 модели к минимальному F1 среди всех моделей
        (model['f1'] / min(m['f1'] for m in models_results.values())) +
        # Нормализация времени обучения: отношение времени обучения модели к минимальному времени среди всех моделей
        (model['train_time'][1] / min(m['train_time'][1] for m in models_results.values())) +
        # Нормализация времени предсказания: отношение времени предсказания модели к минимальному времени среди всех моделей
        (model['pred_time'][1] / min(m['pred_time'][1] for m in models_results.values()))
    ) / 3 # Среднее значение нормализованных критериев

In [47]:
# Вычисляем оценку для каждой модели
scores = {k: score(v) for k, v in models_results.items()}

# Выводим название модели и её оценку
for model, score in scores.items():
    print(f'{model}: {score}')

LogisticRegression: 1.0850579374629767
DecisionTreeClassifier: 4.362984356495228
CatBoostClassifier: 44.146644615990944


In [49]:
# Выбор лучшей модели и тестирование её на тестовой выборке
best_model_name = min(scores, key=scores.get)
best_model = models_results[best_model_name]['model']

# Окончательная оценка на тестовой выборке
predictions, test_time = measure_time(lambda: best_model.predict(X_test))
f1_score = f1_score(y_test, predictions)

print(f'Лучшей моделью оказалась {best_model_name}')
print(f'F1-score на тестовой выборке = {f1_score}')
print(f'Время предсказания на тестовой выборке = {test_time} секунд')

Лучшей моделью оказалась LogisticRegression
F1-score на тестовой выборке = 0.752642953098534
Время предсказания на тестовой выборке = 0.014154672622680664 секунд


## Общие выводы

1. **Загрузка и изучение данных:**

    Были загружены комментарии и оценка их токсичности. Проведён первоначальный анализ данных.


2. **Подготовка данных:**

- Данные очищены и лемматизированы;
- Применена TF-IDF векторизация к обучающей и тестовой выборке.


3. **Обучение моделей:**

    Были обучены несколько моделей машинного обучения: LogisticRegression, DecisionTreeClassifier и CatBoostClassifier. Модели были обучены на подготовленных данных с последующей оценкой их точности.


4. **Сравнение моделей и выбор лучшей:**

    Проведено сравнение моделей на основе метрики accuracy. Выбрана модель, показавшая наилучшие результаты, с минимальным временем обучения и предсказания.


5. **Итоговые выводы:**

    В результате проведённого анализа и моделирования была выбрана лучшая модель (LogisticRegression) с accuracy на тестовой выборке = 0.95, которая способна определить токсичные комментарии.